In [1]:
from platform import python_version

print(python_version())

3.6.13


In [2]:
#https://github.com/david-siqi-liu/yelp-sentiment-analysis/blob/master/notebooks/model_bert.ipynb

In [3]:
%cd /Users/kelsey/yelp

/Users/kelsey/yelp


In [ ]:
!pip install tensorflow==1.15

In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install tensorflow_estimator

In [ ]:
!pip install bert-tensorflow==1.0.1

In [ ]:
!pip install pandas

In [ ]:
!pip install seaborn

In [ ]:
!pip install -U scikit-learn

In [4]:
import random
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [5]:
yelp = pd.read_csv("yelp_clean.csv")

In [6]:
yelp = yelp.fillna("no text")

In [7]:
# train test split
from sklearn.model_selection import train_test_split

x = yelp['text_clean'].tolist()
y = yelp['stars'].tolist()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [8]:
data_train = pd.DataFrame()
data_train['text_clean'] = x_train
data_train['stars'] = y_train

data_test = pd.DataFrame()
data_test['text_clean'] = x_test
data_test['stars'] = y_test

In [9]:
# Set the output directory for saving model file
OUTPUT_DIR = 'bert'

DO_DELETE = True

if DO_DELETE:
    try:
        tf.gfile.DeleteRecursively(OUTPUT_DIR)
    except:
        pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: bert *****


In [10]:
DATA_COLUMN = 'text_clean'
LABEL_COLUMN = 'stars'
label_list = [1, 2, 3, 4, 5]

In [11]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [12]:
train_InputExamples = data_train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                                  text_a = x[DATA_COLUMN], 
                                                                                  text_b = None,
                                                                                  label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = data_test.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                               text_a = x[DATA_COLUMN],
                                                                               text_b = None,
                                                                               label = x[LABEL_COLUMN]), axis = 1)

In [13]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
                               ...                        
44030    <bert.run_classifier.InputExample object at 0x...
44031    <bert.run_classifier.InputExample object at 0x...
44032    <bert.run_classifier.InputExample object at 0x...
44033    <bert.run_classifier.InputExample object at 0x...
44034    <bert.run_classifier.InputExample object at 0x...
Length: 44035, dtype: object

In [14]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  goto place sneak away lunch dinner without kids coming years tried makinos kaizen many others find far better deal better quality variety love ayce sushi

__________
Row 0 - text_b of training set :  None

__________
Row 0 - label of training set :  5


In [15]:
# Not sure if we need this because I think what I have is already tokenized 
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [16]:
# seems to have been split up "extra" somehow? 
print(tokenizer.tokenize(train_InputExamples.iloc[5].text_a))

['new', 'neighborhood', 'cleaner', '##s', 'listen', 'ye', '##lp', '##ers', 'like', 'buying', 'nice', 'well', 'made', 'sometimes', 'expensive', '##des', '##ign', '##er', 'clothes', 'although', 'dislike', 'dry', 'clean', 'tag', 'village', 'east', 'cleaner', '##s', 'never', 'di', '##sa', '##pp', '##oint', '##s', 'oh', 'th', '##ier', 'alterations', 'truly', 'done', 'professionals', 'took', 'brand', '##ne', '##w', 'bet', '##tie', 'page', 'dress', 'get', 'shortened', 'don', '##t', 'judge', 'lo', '##l', 'got', 'back', 'perfect', 'seam', '##st', '##ress', 'listened', 'exactly', 'wanted', 'made', 'changes', 'asked']


In [17]:
MAX_SEQ_LENGTH = 350

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list,
                                                                  MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, 
                                                                MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 44035


INFO:tensorflow:Writing example 0 of 44035


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:label: 4 (id = 3)


INFO:tensorflow:Writing example 10000 of 21689


INFO:tensorflow:Writing example 10000 of 21689


INFO:tensorflow:Writing example 20000 of 21689


INFO:tensorflow:Writing example 20000 of 21689


In [18]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    bert_module = hub.Module(
        BERT_MODEL_HUB,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [19]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:
        (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
        
        train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
        def metric_fn(label_ids, predicted_labels):
            accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
            true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
            true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
            false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
            false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
            return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

        eval_metrics = metric_fn(label_ids, predicted_labels)

        if mode == tf.estimator.ModeKeys.TRAIN:
            return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
        else:
            return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
        (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
        
        predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [20]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 8
# LEARNING_RATE = 2e-5
LEARNING_RATE = 5e-6
NUM_TRAIN_EPOCHS = 2.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
# SAVE_CHECKPOINTS_STEPS = 300
# SAVE_SUMMARY_STEPS = 100
SAVE_CHECKPOINTS_STEPS = 15000
SAVE_SUMMARY_STEPS = 1500

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)
print(num_warmup_steps)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

11008
1100


In [21]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'bert', '_tf_random_seed': None, '_save_summary_steps': 1500, '_save_checkpoints_steps': 15000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb613dc3710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'bert', '_tf_random_seed': None, '_save_summary_steps': 1500, '_save_checkpoints_steps': 15000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb613dc3710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [24]:
 #Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 0 into bert/model.ckpt.


INFO:tensorflow:Saving checkpoints for 0 into bert/model.ckpt.


INFO:tensorflow:loss = 1.5568897, step = 1


INFO:tensorflow:loss = 1.5568897, step = 1


KeyboardInterrupt: 

In [ ]:
# A method to get predictions
def getPrediction(in_sentences):
    #A list to map the actual labels to the predictions
    labels = [1, 2, 3, 4, 5]

    #Transforming the test data into BERT accepted form
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
    #Creating input features for Test data
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

    #Predicting the classes 
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentenc

In [ ]:
predictions = estimator.predict(input_fn=val_input_fn)

In [ ]:
y_test_pred = [(p['probabilities'], p['labels']) for p in predictions]

In [ ]:
visualization.confusion_heat_map(y_test,
                                 y_test_pred,
                                 normalize='true',
                                 fmt='.1%',
                                 labels=set(y_test))

f1_train, f1_test = metrics.f1_score(y_train, y_train_pred), metrics.f1_score(y_test, y_test_pred)

print("Macro F1 Scores: \n Training: {0:.3f} \n Testing: {1:.3f}\n\n".format(f1_train, f1_test))